In [ ]:
import pandas as pd
import glob
import re
import math
import urllib.request
from bs4 import BeautifulSoup
from operator import itemgetter
import csv
import os

In [ ]:
def news_deserts_csv_df(keywords_list):
#     filepath = "/Users/tdt62/Desktop/test_data/*clean*"
    filepath = "/Users/tdt62/Desktop/test_data/2019*stream_1*"

    list_ = []

    # Takes all of the csv file and makes one big dataframe
    for name in glob.glob(filepath):
        df = pd.read_csv(name,index_col=None, header=0, sep='\t', names=["TweetID", "Timestamp", "Full_Text", "In_Reply_To_User_ID", "User_ID", "User_Name", "User_Screen_Name", "Coordinates", "Place", "Bounding_Box", "Quoted_Status_ID", "Retweeted_Status", "Hashtags", "URLs", "User_Mentions", "Media", "Language"])
        list_.append(df)

    # filepath = "/projects/canis/news_deserts/twitter/data/2018_11_06*clean*"
    # #filepath = "/Users/tdt62/Desktop/test_data/2018_11_0[0-5]*clean*"
    #
    # # Takes all of the csv file and makes one big dataframe
    # for name in glob.glob(filepath):
    #     df = pd.read_csv(name,index_col=None, header=0, delimiter='\t')
    #     list_.append(df)

    # Makes the big df in memory
    frame = pd.concat(list_, axis = 0, ignore_index = True)
    frame.fillna("NA", inplace=True)
    
    def check_keywords(x):
        for keyword in keywords:
            if(keyword.lower() in x.lower()):
                return True
        return False
    
        # Make a seperate column which is True if it has the keyword, False if not
    frame['has_keyword'] = list(map(lambda x: check_keywords(x), frame['Full_Text']))
    
    # Sort based on True/False values from above
    keywords_df = frame.loc[frame['has_keyword'] == True]
    return keywords_df

def total_tweets(df):

    # Gets the total number of tweets
    total_tweets_num = df.shape[0]  # gives number of row count
    return total_tweets_num

def unique_tweets(df):

    def run_isdigit(x):
        if math.isnan(x):
            return False
        else:
            return x.isdigit()

    # Gets the unique tweets i.e. no retweeted status
    df['rt_isdigit'] = list(map(lambda x: x.isdigit(), df['Retweeted_Status']))
    unique_df = df.loc[df['rt_isdigit'] == False]
    unique_tweet_num = total_tweets(unique_df)
    return unique_tweet_num

def unique_domains(df):
    
    # Checks if URLs is empty or not
    def empty_list_check(x):
        if(x != '[]'):
            return True
        else:
            return False

    def get_domains(full_domain):
        if(isinstance(full_domain, str)):
            m = full_domain.split("//")[-1].split("/")[0].split('?')[0]
            return m

        else:
            return full_domain


    # Gets the tweets that have URLs in them i.e. URLs column is not '[]'
    df['domains'] = list(map(lambda x: empty_list_check(x), df['URLs']))

    # Gets the domains to be only the domains i.e. www.xyz.com
    df['actual_domain'] = list(map(lambda x: get_domains(x), df['URLs']))

    # Creates a df that only has domains in the tweet
    unique_df = df.loc[df['domains'] == True]

    # Get the number of domains
    unique_domains = len(unique_df.actual_domain.unique())

    domains_df = unique_df

    return unique_domains, domains_df

def unique_users(df):
    unique_users = len(df.User_ID.unique())
    return unique_users

def original_content_user(df):

    def run_isdigit(x):
        if math.isnan(x):
            return False
        else:
            return x.isdigit()

    # Gets the unique tweets i.e. no retweeted status
    df['rt_isdigit'] = list(map(lambda x: x.isdigit(), df['Retweeted_Status']))
    unique_df = df.loc[df['rt_isdigit'] == False]
    unique_users = len(unique_df.User_ID.unique())
    return unique_users

def top_news_domains():

    #specify the url
    news_source = "https://www.alexa.com/topsites/category/Top/News"

    #Query the website and return the html to the variable 'page'
    page = urllib.request.urlopen(news_source)

    #Parse the html in the 'page' variable, and find all of the website domains
    soup = BeautifulSoup(page, features="html.parser")
    links = soup.find_all("a")
    link_list = []
    actual_news_list = []
    links_dict = {}
    for link in links:
        link_list.append(link.get("href"))
    i = 0
    for link in link_list:
        if link:
            if "siteinfo/" in link:
                news_link = link.split("/siteinfo/")[1]
                links_dict[news_link] = 0
                actual_news_list.append(news_link)

    return actual_news_list, links_dict

def get_top_news_domains(df, news_list, news_dict, top_val):

    # Function to iterate over the dataframe
    def iterate_domains_dict(x):
        for element in x.split():
            if element in news_list:
                news_dict[x] += 1
        return 0


    top_list = []

    # Iterates over the domain column
    list(map(lambda x: iterate_domains_dict(x), df['actual_domain']))

    # Gets the sorted news_list in descending order
    sorted_news_list = (list(sorted(news_dict.items(), key=itemgetter(1), reverse=True)))

    # Gets the top 20 results of the sorted list
    sorted_news_list = sorted_news_list[0:top_val]

    # Stores all the names in a list, so we can read correctky
    for item in sorted_news_list:
        top_list.append(item[0])

    return top_list, sorted_news_list

def get_top_domains(df, top_val):
    top_list = []
    domain_dict = {}

    def iterate_domains(x):
        domain_list = list(x.split())
        for element in domain_list:
            if element in domain_dict:
                domain_dict[element] += 1
            else:
                domain_dict[element] = 1

    # Create the hashtag dict
    list(map(lambda x: iterate_domains(x), df['actual_domain']))

    # Gets the sorted news_list in descending order
    sorted_domain_list = (list(sorted(domain_dict.items(), key=itemgetter(1), reverse=True)))

    # Gets the top 20 results of the sorted list
    sorted_domain_list = sorted_domain_list[0:top_val]

    # Stores all the names in a list, so we can read correctky
    for item in sorted_domain_list:
        top_list.append(item[0])

    return top_list, sorted_domain_list

def get_number_of_domains(df):
    total_number_domains = len(df.actual_domain)
    return total_number_domains

def count_total_news(news_dict):
    total = sum(list(news_dict.values()))
    return total

def make_domain_csv(top_news_dict):
    with open('news_domains.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in top_news_dict.items():
            writer.writerow([key, value])

def make_hashtag_csv(hashtag_dict):
    with open('top_hashtags.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in top_news_dict.items():
            writer.writerow([key, value])

def make_domain_csv(domain_dict):
    with open('top_domains.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in top_news_dict.items():
            writer.writerow([key, value])

def hashtags(df, top_val):

    top_list = []
    hashtag_dict = {}

    def iterate_hashtags(x):
        hashtag_list = list(x.split(', '))
        for element in hashtag_list:
            if element.strip('[]') in hashtag_dict and x != '[]':
                hashtag_dict[element.strip('[]')] += 1
            else:
                hashtag_dict[element.strip('[]')] = 1

    # Create the hashtag dict
    list(map(lambda x: iterate_hashtags(x), df['Hashtags']))

    # Gets the sorted news_list in descending order
    sorted_news_list = (list(sorted(hashtag_dict.items(), key=itemgetter(1), reverse=True)))

    # Gets the top 20 results of the sorted list
    sorted_news_list = sorted_news_list[0:top_val]

    # Stores all the names in a list, so we can read correctky
    for item in sorted_news_list:
        top_list.append(item[0])

    return top_list, sorted_news_list

def top_user_mentions(df, top_val):

    top_users_list = []
    hashtag_dict = {}

    def iterate_user_mentions(x):

        hashtag_list = list(x.split())
        for element in hashtag_list:
            if element.strip('{}:,').isdigit():
                continue
            elif element.strip('{}:') in hashtag_dict and element != '{}' and not element.strip('{}:').isdigit():
                hashtag_dict[element.strip('{}:')] += 1
            else:
                hashtag_dict[element.strip('{}:')] = 1

    # Create the hashtag dict
    list(map(lambda x: iterate_user_mentions(x), df['User_Mentions']))

    # Gets the sorted news_list in descending order
    sorted_news_list = (list(sorted(hashtag_dict.items(), key=itemgetter(1), reverse=True)))

    # Gets the top 20 results of the sorted list
    sorted_news_list = sorted_news_list[0:top_val]

    # Stores all the names in a list, so we can read correctky
    for item in sorted_news_list:
        top_users_list.append(item[0])

    return top_users_list, sorted_news_list

In [ ]:
# native_df = news_deserts_csv_df()
# native_df

In [ ]:
keywords = ["Midterm","Vote","Politics","District","Senator","Congress","elect","Representative","Sen","Rep",
            "Republican","Democrat","Dem","Rep","Gov","Debates","GOP","Ballot","Register",
            "Incumbent","Delegate","Potus","Scotus","Supreme court","local news"]

news_df = news_deserts_csv_df(keywords)
# news_df = native_vote_csv_df()


In [ ]:
news_df

In [ ]:
total_tweets(news_df)

In [ ]:
keywords = ["Midterm","Vote","Politics","District","Senator","Congress","elect","Representative","Sen","Rep",
            "Republican","Democrat","Dem","Rep","Gov","Debates","GOP","Ballot","Register",
            "Incumbent","Delegate","Potus","Scotus","Supreme court","local news"]

# Create big df for manipulating the data
news_df = news_deserts_csv_df(keywords)

# Get the count of unique domains and the df associated
unique_domains_count, domains_df = unique_domains(news_df)

# Total number of tweets 
count_row = total_tweets(news_df)

# Unique number of tweets
unique_tweets_count = unique_tweets(news_df)

# Unique users based on UserID
unique_users_count = unique_users(news_df)

# Original Content creater count
original_content = original_content_user(news_df)

# Generates the list and dict associated with the domains
top_news_list, top_news_dict = top_news_domains()

# Gets the top 10 news domains
top_news_list, sorted_top_news_list = get_top_news_domains(domains_df, top_news_list, top_news_dict, 10)

# Gets top 20 domains
top_domain_list, sorted_top_news_list = get_top_domains(domains_df, 20)

# Gets the total number of domains mentioned
total_domains = get_number_of_domains(domains_df)

# Count total number of news domains
total_news_domains = count_total_news(top_news_dict)

# Generate a CSV for the domain and it's occurence
make_domain_csv(top_news_dict)

# Gets the list for top 20 hashtags
top_hashtags, hashtag_sorted_list = hashtags(news_df, 20)

# Get the top 20 user mentions
top_users_list, sorted_news_list = top_user_mentions(news_df, 20)

# Prints each value
print("Total Number of Tweets: {}".format(count_row))
print("Unique Tweets: {}".format(unique_tweets_count))
print("Unique Domains: {}".format(unique_domains_count))
print("Unique Users: {}".format(unique_users_count))
print("Original Content Users: {}".format(original_content))
print("Total Number of Domains: {}".format(total_domains))
print("Total Number of News Domains: {}\n".format(total_news_domains))
print("Top 10 News Domains: {}\n".format(top_news_list))
print("Top 20 Hashtags: {}\n".format(top_hashtags))
print("Top 20 Domains: {}\n".format(top_domain_list))
print("Top 20 User Mentions: {}\n".format(top_users_list))

In [29]:
### LOCATION DATA SCRIPTS ###
import os
import sys
import csv
import argparse
import pandas as pd
from geopy.geocoders import Nominatim



# https://simplemaps.com/data/us-cities

# twitter.com/anyuser/status/541278904204668929

hashtag_state_full =  ['\'Alabama\'', '\'Alaska\'', '\'Arizona\'', '\'Arkansas\'', \
'\'California\'', '\'Colorado\'', '\'Connecticut\'', '\'Delaware\'', '\'Florida\'', \
'\'Georgia\'', '\'Hawaii\'', '\'Idaho\'', '\'Illinois\'' , '\'Indiana\'' , '\'Iowa' , \
'\'Kansas\'' , '\'Kentucky\'' , '\'Louisiana\'' , '\'Maine\'' , '\'Maryland\'' , \
'\'Massachusetts\'' , '\'Michigan\'' , '\'Minnesota\'' , '\'Mississippi\'' , \
'\'Missouri\'' , '\'Montana\'' '\'Nebraska\'' , '\'Nevada' , '\'NewHampshire\'' , \
'\'NewJersey\'' , '\'NewMexico\'' , '\'NewYork\'' , '\'NorthCarolina\'' , \
'\'NorthDakota\'' , '\'Ohio' , '\'Oklahoma\'' , '\'Oregon\'' , '\'Pennsylvania\'', \
'\'RhodeIsland\'', '\'SouthCarolina\'' , '\'SouthDakota\'' , '\'Tennessee\'' , \
'\'Texas\'' , '\'Utah\'' , '\'Vermont\'' , '\'Virginia\'' , '\'Washington\'' , \
'\'WestVirginia\'' , '\'Wisconsin\'', '\'Wyoming\'']

hashtag_state_abbr = ['\'AL\'', '\'AK\'', '\'AZ\'', '\'AR\'', '\'CA\'', '\'CO\'', \
    '\'CT\'', '\'DE\'', '\'FL\'', '\'GA\'', '\'HI\'', '\'ID\'', '\'IL\'', '\'IN\'', \
    '\'IA\'', '\'KS\'', '\'KY\'', '\'LA\'', '\'ME\'', '\'MD\'', '\'MA\'', '\'MI\'', \
    '\'MN\'', '\'MS\'', '\'MO\'', '\'MT\'', '\'NE\'', '\'NV\'', '\'NH\'', '\'NJ\'', \
    '\'NM\'', '\'NY\'', '\'NC\'', '\'ND\'', '\'OH\'', '\'OK\'', '\'OR\'', '\'PA\'', \
    '\'RI\'', '\'SC\'', '\'SD\'', '\'TN\'', '\'TX\'', '\'UT\'', '\'VT\'', '\'VA\'', \
    '\'WA\'', '\'WV\'', '\'WI\'', '\'WY\'']



city_state_abbr = []
city_state_full = []
city_list_indicies = []

def make_domain_csv(domain_dict):
    with open('top_domains.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        for key, value in top_news_dict.items():
            writer.writerow([key, value])
            
def city_master_list():
    
    df = pd.read_csv('/Users/tdt62/Desktop/GraduateResearch/scripts/location_data/uscitiesv1.4.csv', index_col=None, header=0)

    return df


def find_city_state(TSV_file, TSV_output_file, df):
    
    city_state_abbr = set(zip(df['lat'], zip(df['lng'], zip(df['state_id'], zip(df['city'], df['state_name'])))))
    
    twitter_df = pd.read_csv(TSV_file, index_col=None, header=0, delimiter='\t')    
    
    location_df = pd.DataFrame(columns=['Tweet ID', 'Name Place', 'City ID'])
    
    def location_csv(twitter_df):
        coordinates = []
        ctyid = []
        places = []
        has_location = []
        for city in city_state_abbr:
            place = str(', '.join(city[1][1][1]))
            if str(city[0]) in twitter_df['Full_Text'] or place in twitter_df['Full_Text']:
                has_location.append(True)
                places.append(place)
                ctyid.append(city[1][1][0])
                coordinates.append((city[0], city[1][0]))
                
        return has_location, places, ctyid, coordinates 

    import time
    start = time.time()
    twitter_df = twitter_df[0:10]
    twitter_df['Location'], twitter_df['Place'], twitter_df['City ID'], twitter_df['Coordinates'] = list(zip(*twitter_df[0:10].apply(location_csv, axis=1)))
    end = time.time()
    print("Time: {}".format(end - start))
    twitter_df.to_csv(TSV_output_file, sep='\t')
    return twitter_df

    
master_list = city_master_list()
# master_list
frame = find_city_state("/Users/tdt62/Desktop/test_data/2018_11_05_06_stream_1_clean.csv", "/Users/tdt62/Desktop/GraduateResearch/scripts/location_data/test.csv", master_list)
keywords_df = frame.loc[frame['Location'] == True]
keywords_df


# def parse_args():
#     '''
#     Parses the command line arguments for the script

#     Returns
#     -------

#     args : object
#            Python arg parser object
#     '''
#     parser = argparse.ArgumentParser(description='Process some integers.')

#     parser.add_argument('TSV_file', type=str, help='Twitter stream TSV file to find city states.')

#     parser.add_argument('TSV_output_file', type=str, help='Output file with found city states.')

#     args = parser.parse_args()
#     return args

# def main():
#     args = parse_args()

#     city_list_indicies = city_master_list()

#     find_city_state(args.TSV_file, args.TSV_output_file, city_list_indicies)

# if __name__ == "__main__":
#     main()


('Upland', 'California')
('Upland', 'California')
('Upland', 'California')
('Upland', 'California')
('Upland', 'California')
('Upland', 'California')
('Upland', 'California')
('Upland', 'California')
('Upland', 'California')
('Upland', 'California')


TypeError: zip argument #1 must support iteration

In [24]:
print(frame.keys())

Index(['Unnamed: 0', 'TweetID', 'Timestamp', 'Full_Text',
       'In_Reply_To_User_ID', 'User_ID', 'User_Name', 'User_Screen_Name',
       'Coordinates', 'Place', 'Bounding_Box', 'Quoted_Status_ID',
       'Retweeted_Status', 'Hashtags', 'URLs', 'User_Mentions', 'Media',
       'Language', 'Location', 'Name Place', 'City ID'],
      dtype='object')


In [ ]:
filepath = "/Users/tdt62/Desktop/test_data/2018_12*stream_1*"

list_ = []

# Takes all of the csv file and makes one big dataframe
for name in glob.glob(filepath):
    with open(name, newline='') as csvfile:
        dialect = csv.Sniffer().sniff(csvfile.read(10))
        csvfile.seek(0)
        reader = csv.reader(csvfile, dialect)
        print(dialect.delimiter)

#     df = pd.read_csv(name,index_col=None, header=0, delimiter='\t')
#     list_.append(df)


    # ... process CSV file contents here .